In [ ]:
import pandas as pd
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

Pour voir toutes les colonnes `set_option` au cas ou Google Colab n'affiche que les premières et dernières colonnes.

Aujourd'hui seules les bibliothèques utiles seront importées.

In [ ]:
housing_train = pd.read_csv('HousingPriceTrain.csv')
housing_test = pd.read_csv('HousingPriceTest.csv')

Le premier sert à l'entraînement (contenant la colonnes Labels) et le second au test.  
C'est un gros dataset par le nombre de colonnes.

In [ ]:
housing_train.shape, housing_test.shape
# ((1460, 81), (1459, 80))

Combien de colonnes n'ont pas de valeurs ?

In [ ]:
null_value_cols = [col for col in housing_train.columns if housing_train[col].isnull().any()]

housing_train[null_value_cols].isnull().sum()
"""
LotFrontage      259
Alley           1369
MasVnrType       872
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
"""

Ayant beaucoup de valeurs manquante, '`Alley`', '`PoolQC`', '`Fence`' et '`MiscFeature`' sont inutiles, ainsi que '`MasVnrType`' et '`ireplaceQu`' ayant environ 50% de valeurs manquates.

In [ ]:
housing_train = housing_train.drop(columns=['Alley', 'MasVnrType' 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature', 'LotFrontage'])
housing_test = housing_test.drop(columns=['Alley', 'MasVnrType', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature'])

Vérifier le type de données de toutes les colonnes à valeurs null ; pour une colonne catégorielle ou numérique, la stratégie diffère.

In [ ]:
housing_train[null_value_cols].dtypes
"""
LotFrontage     float64
MasVnrArea      float64
BsmtQual         object
BsmtCond         object
BsmtExposure     object
BsmtFinType1     object
BsmtFinType2     object
Electrical       object
GarageType       object
GarageYrBlt     float64
GarageFinish     object
GarageQual       object
GarageCond       object
"""

 '`LotFrontage`' manque de beaucop de valeurs. En ne supprimant que les lignes manquantes, la perte de lignes et trop importante ; en supprimant la colonne, la perte de 140 lignes est acceptable.

In [ ]:
housing_train = housing_train.drop(columns='LotFrontage')

fresh_housing_train = housing_train.dropna()
fresh_housing_train.shape   # (1338, 74)

La forme/shape du DataFrame est passée de 1460 à 1338 sans valeurs null ou de valeurs manquantes.

In [ ]:
low_cardinality_col = [col for col in fresh_housing_train.columns
                       if fresh_housing_train[col].dtype == object
                       and fresh_housing_train[col].nunique() < 10]
high_cardinality_col = [col for col in fresh_housing_train.columns
                        if fresh_housing_train[col].dtype == object
                        and fresh_housing_train[col].nunique() >= 10]
num_col = [col for col in fresh_housing_train.columns if fresh_housing_train[col].dtype in [int, float]]

Séparation des colonnes en 3 catégories , n'ayant pas le même pré-traitement :
* '`low_cardinality_col`' pour les colonnes catégorielles de moins de 10 valeurs uniques ;
* '`high_cardinality_col`' celles présentant un grand nombre de catégories (+10) ;
*  '`num_col`' pour les colonnes numériques.

In [ ]:
new_housing_train = pd.concat([fresh_housing_train[low_cardinality_col], fresh_housing_train[high_cardinality_col], fresh_housing_train[num_col]], axis=1)

La colonne '`id`' est inutile et supprimmmer '`LostFrontage`'.  
Recombiner les colonnes.

In [ ]:
new_housing_train = new_housing_train.drop(columns='ID')
housing_test = housing_test.drop(columns=['LotFrontage', 'ID'])

Appliquer à l’ensemble de test tous les traitements préalables effectués pour l’ensemble d’entraînement train.

In [ ]:
null_value_cols = [col for col in housing_test.columns if housing_test[col].isnull().any()]
housing_test[housing_test.isnull().any(axis=1)][null_value_cols]
fresh_housing_test = housing_test.dropna()
low_cardinality_col = [col for col in fresh_housing_test.columns
                       if fresh_housing_test[col].dtype == object
                       and fresh_housing_test[col].nunique() < 10]
high_cardinality_col = [col for col in fresh_housing_test.columns
                        if fresh_housing_test[col].dtype == object
                        and fresh_housing_test[col].nunique() >= 10]
num_col = [col for col in fresh_housing_test.columns if fresh_housing_test[col].dtype in [int, float]]
new_housing_test = pd.concat([fresh_housing_test[low_cardinality_col], fresh_housing_test[high_cardinality_col],
                             fresh_housing_test[num_col]], axis=1)

Copier l'ensemble d'entraînement et de test et poursuivre le traitement.

In [ ]:
copy_of_housing_train = new_housing_train.copy()
copy_of_housing_test = new_housing_test.copy()

Traiter ou encoder les valeurs catégorielles.

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
cat_col = low_cardinality_col + high_cardinality_col
for col in cat_col:
  new_housing_train[col] = le.fit_transform(new_housing_train[col])
for col in cat_col:
  new_housing_test[col] = le.fit_transform(new_housing_test[col])

Les ensembles d'entraînement et de test sont parfaits.  
Diviser le set d'entraînement en set d'entraînement + set de validation.  
Utiliser l'ensemble de test que pour les tests.

In [ ]:
from sklearn.model_selection import train_test_split
y = new_housing_train['SalePrice']
X = new_housing_train.drop(columns='SalePrice')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=766)
X_train.shape, X_test.shape   # ((1070, 72), (268, 72))

Les colonnes numéiques ne sont pas normalisés ; le '`DecisionTreeRegressor`' et '`le RandomForestRegressor`' fonctionnent bienn avec ce type de données.

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
dtr = DecisionTreeRegressor()
rfr = RandomForestRegressor()
dtr.fit(X_train, y_train)
dtr_prediction = dtr.predict(X_test)
rfr.fit(X_train, y_train)
rfr_prediction = rfr.predict(X_test)

Quel modèle fonctoinne mieux ?

In [1]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
dtr_error = mean_absolute_error(y_test, dtr_prediction)
dtr_error_percentage = mean_absolute_percentage_error(y_test, dtr_prediction)
dtr_error, dtr_error_percentage*100   # (26516.899253731342, 14.019438882578578) C'est bien.

NameError: name 'y_test' is not defined

In [ ]:
rfr_error = mean_absolute_error(y_test, rfr_prediction)
rfr_error_percentage = mean_absolute_percentage_error(y_test, rfr_prediction)
rfr_error, rfr_error_percentage*100   # (18436.71156716418, 10.196052787219896) C'est mieux.

Sansparamètre, les 2 modèles fonctionnent bien ; il y a eu un bon pré-traitement des données.  
L'Analyse Exploratoire de Données est très importante.